In [ ]:
#json 파일 yolo 포맷에 맞게 각 image 별 txt 파일로 변환 
def make_yolo_format_by_json_information(src_path = '[라벨]상표이미지_valid',
                                 IMG_DIR = '[원천]상표이미지_valid',
                                 dst_path='valid',
                                 mapping_path = 'mapping.txt',
                                 is_subfolder_exist = True,
                                 MAX_CATEGORY_COUNT = 5000, #샘플수량 <= each_category_count
                                 value_key_reverse = True):

    import json
    from PIL import Image

    import pandas as pd
    
    category_id_list = os.listdir(IMG_DIR)

    json_list = os.listdir(src_path)
    mapping = {} 
    
    #소분류 392
    with open(mapping_path,'r') as f:
        lines = f.readlines()
    
    for line in lines:
            # Split the line into a key and a value
        value, key = line.strip().split(',') 

        if value_key_reverse:
            mapping[value] = key
        else:
            # Insert the key and value into the dictionary
            mapping[key] = value  # convert the key to int because your keys are integers

    for json_file in json_list:
        #if json_file[:6] != '270508':
            #continue
        
        #skip category
        if json_file[:6] not in mapping:
            continue 


        json_file_path = os.path.join(src_path,json_file)
        
        # Load your json file
        with open(json_file_path, 'r') as f:
            data = json.load(f)
         
        #중복 id체크용 리스트   
        check_overlapped_list = [] 
        #카테고리 카운트용 
        category_count = 0   
        
        one_over_count = 0  
        zero_less_count = 0 
         
        for annotation in data['annotations']:

            
            
            json_folder = json_file.split('.')[0]
            image_id = annotation['image_id']
            
            #중복 id 체크 
            if image_id in check_overlapped_list:
                continue
            
            check_overlapped_list.append(image_id) 
            
            #서브 폴더 있을시 
            if is_subfolder_exist:
                img_path = os.path.join(IMG_DIR,json_folder,json_folder.split('_')[0] + '_' + annotation['image_id'] + '.jpg')
            else:
                img_path = os.path.join(IMG_DIR,json_folder.split('_')[0] + '_' + annotation['image_id'] + '.jpg')

            if not os.path.exists(img_path):
                continue
            
            try:
                # Open and close the image to check if it's corrupted
                try:
                    img = Image.open(img_path)  
                except Exception:
                    continue
                img.verify()
                img.close()             
            except (IOError, SyntaxError) as e:
                print(f"Bad file: {img_path}\n{e}")
                continue

            if img is None:

                continue 
            
          
            img_width, img_height = img.size
            
            image_name = json_folder.split('_')[0] + '_' + annotation['image_id'] + '.jpg'
            #print(image_name)
            #print(f"{dst_path}/{json_file.split('.')[0]}/{image_name.split('.')[0]}.txt")
            
            # Convert bbox values to image size
            bbox = [annotation['bbox'][0],  # xmin
                    annotation['bbox'][1] ,  # ymin
                    annotation['bbox'][2] + annotation['bbox'][0],  # xmax
                    annotation['bbox'][3] + annotation['bbox'][1]]  # ymax
            
            #데이타가 안맞는게 있슴... 
            if bbox[0] < 0 or bbox[1] < 0 or bbox[2] < 0 or bbox[3] < 0 or bbox[3] < bbox[1] or bbox[2] < bbox[0]:
              
                continue
            
            if (img_width * img_width) >= 170000000: # 최대 픽셀값 
            
           
                continue
        
            #if is_subfolder_exist:
                #dst_txt_path =  f"{dst_path}/{json_file.split('.')[0]}/{image_name.split('.')[0]}.txt"
            #else:
            dst_txt_path =  f"{dst_path}/{image_name.split('.')[0]}.txt"
             
            # Bounding box coordinates
            xmin = annotation['bbox'][0]
            ymin = annotation['bbox'][1]
            xmax = annotation['bbox'][2] + xmin
            ymax = annotation['bbox'][3] + ymin
                    
            # Convert to Darknet format
            x_center = ((xmin + xmax) / 2.0) / img_width
            y_center = ((ymin + ymax) / 2.0) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height 
            
            #print(x_center,y_center,width,height)   
            
            #1넘어가는 값 있는지 체크
            if x_center > 1 or y_center > 1 or width > 1 or height > 1:
                one_over_count += 1
                print('1넘어가는 값 = ',one_over_count)
                continue
            #0미만인 값 있는지 체크
            if x_center < 0 or y_center < 0 or width < 0 or height < 0:
                zero_less_count += 1
                print('0미만인 값 = ',zero_less_count)
                continue  
   
            #with 구문에는 continue 쓰면 안됨   
            with open(dst_txt_path, 'w') as file:

                # Class index
                class_index = mapping[json_file.split('_')[0]]

                # Write to the file
                file.write(f"{class_index} {x_center} {y_center} {width} {height}\n")
                 
            
            
            category_count += 1 
            #각 카테고리 수량 max 체크
            if category_count >= MAX_CATEGORY_COUNT:
                print(json_file,' ',category_count)
                break 

make_yolo_format_by_json_information()
